# VARIATIONAL QUANTUM CLASSIFIER

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from qiskit import transpile
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime.fake_provider import FakeVigoV2
from qiskit.primitives import StatevectorSampler
from qiskit.circuit.library import ZFeatureMap, ZZFeatureMap, PauliFeatureMap
from qiskit.circuit.library import EfficientSU2, RealAmplitudes
from matplotlib import pyplot as plt
from scipy.optimize import minimize
from sklearn.metrics import accuracy_score,precision_score,recall_score, f1_score, log_loss,confusion_matrix


In [ ]:
label="Outcome"

### Caricamento del dataset CSV

In [ ]:
path_file= "../dataset/final_diabetes_dataset_5.csv"
df=pd.read_csv(path_file)

###  Funzione di normalizzazione per Feature Map

Definisce due funzioni:
* `rescalerZFeatureMap`: normalizza le feature nel range $ [0,\pi ]$.

* `rescalerZZFeatureMap`: normalizza le feature nel range $ [0,1] $.


In [ ]:
def rescalerZFeatureMap(df):
    scaler=MinMaxScaler(feature_range=(0,np.pi))
    columns_to_scale = df.columns.difference([label])
    df_scaled_part = pd.DataFrame(scaler.fit_transform(df[columns_to_scale]), columns=columns_to_scale)
    df_scaled = pd.concat([df_scaled_part, df[label]], axis=1)
    df_scaled = df_scaled[df.columns]
    return df_scaled

def rescalerZZFeatureMap(df):
    scaler = MinMaxScaler(feature_range=(0, 1))
    columns_to_scale = df.columns.difference([label])
    df_scaled_part = pd.DataFrame(scaler.fit_transform(df[columns_to_scale]), columns=columns_to_scale)
    df_scaled = pd.concat([df_scaled_part, df[label]], axis=1)
    df_scaled = df_scaled[df.columns]
    return df_scaled

In [ ]:
#df=rescalerZZFeatureMap(df)
df=rescalerZFeatureMap(df)

### Separazione delle feature e dell’etichetta

In [ ]:
X = df.drop(label, axis=1)
y = df[label]

### Suddivisione del dataset in train e test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
X_train_new=X_train.values.tolist()
X_test_new=X_test.values.tolist()
y_train_new=y_train.tolist()
y_test_new=y_test.tolist()

### Circuito per Encoding

In [ ]:
num_features = df.shape[1]-1

### Costruzione della Feature Map
Definisce il circuito di encoding `feature_map` usando una `ZFeatureMap` o `PauliFeatureMap` o `ZZFeatureMap` .

In [ ]:
#feature_map = PauliFeatureMap(feature_dimension=num_features, reps=1)
#feature_map = ZZFeatureMap(feature_dimension=num_features, reps=1)
feature_map = ZFeatureMap(feature_dimension=num_features, reps=1)
feature_map.barrier()
feature_map.decompose().draw(output="mpl", fold=20)

### Definizione dell’Ansatz parametrico
Crea un circuito ansatz usando `EfficientSU2`o in alternativa `RealAmplitudes`.

In [ ]:
ansatz=EfficientSU2(num_features,reps=2)
#ansatz=RealAmplitudes(num_features,reps=2)
ansatz.barrier()
ansatz.decompose().draw(output="mpl", fold=20)

### Combinazione di encoding e ansatz

In [ ]:
ad_hoc_circuit = feature_map.compose(ansatz)
ad_hoc_circuit.measure_all()
ad_hoc_circuit.decompose().draw(output="mpl", style="iqp")

### Assegnazione parametri e interpretazione dei bitstring
definisce:
* `circuit_instance`: assegna i valori classici (feature dell'input) alla feature map e i parametri variabili $θ$ alla ansatz, unendoli nel circuito finale.
* `interpreter`: interpreta un risultato quantistico (btstring misurato) come un'etichetta di classe.
* `label_probability`: calcola la probabilità che un istanza rappresenti la classe 0 o 1 .

In [ ]:
def circuit_instance(tuple, variational):
    # tuple: una tupla
    # variational: parametri dell'ansatz (theta)
    parameters = {}
    for i, p in enumerate(feature_map.ordered_parameters):
        parameters[p] =tuple[i]
    for i, p in enumerate(ansatz.ordered_parameters):
        parameters[p] = variational[i]
    return ad_hoc_circuit.assign_parameters(parameters)

def interpreter(bitstring):
    hamming_weight = sum(int(k) for k in list(bitstring))
    return (hamming_weight) % 2

def label_probability(results):
    shots = sum(results.values())
    probabilities = {0: 0, 1: 0}
    for bitstring, counts in results.items():
        label = interpreter(bitstring)
        probabilities[label] += counts / shots
    return probabilities

### Probabilità di classificazione (ideale e rumorosa)
definisce:
* `classification_probability`: esegue la classificazione quantistica in ambiente ideale (senza rumore).

* `classification_probability_with_noisy`: simula la classificazione con rumore, per imitare un ambiente quantistico reale.

In [ ]:
def classification_probability(data, variational):
    circuits = [circuit_instance(tupla, variational) for tupla in data]
    sampler = StatevectorSampler()
    results = sampler.run(circuits).result()
    classification = [label_probability(results[i].data.meas.get_counts()) for i, c in enumerate(circuits)]

    return classification

def classification_probability_with_noisy(data, variational):
    circuits = [circuit_instance(tupla, variational) for tupla in data]

    fake_backend = FakeVigoV2()
    sim = AerSimulator.from_backend(fake_backend)

    transpiled_qc = transpile(circuits, sim)
    results = sim.run(transpiled_qc).result()

    classifications = []

    for result in results.get_counts():
        classifications.append(label_probability(result))

    return classifications

###  Definizione della funzione di costo per l'ottimizzazione
definisce:

* `cost_function`: usa `classification_probability()` per ottenere le probabilità predette da ciascun circuito e successivamente trasfora queste probabilità in una forma accettata da log_loss di sklearn.

* `cost_function_with_noisy`: usa `classification_probability_with_noisy()` calcola la stessa funzione di costo, ma in presenza di rumore quantistico.


In [ ]:
def cost_function(data, labels, variational):
    classifications = classification_probability(data, variational)
    cost=log_loss(y_true=labels,y_pred=[[p[0],p[1]] for p in classifications])
    return cost

def cost_function_with_noisy(data, labels, variational):
    classifications = classification_probability_with_noisy(data, variational)
    cost=log_loss(y_true=labels,y_pred=[[p[0],p[1]] for p in classifications])
    return cost

def objective_function(variational):
    #return cost_function_with_noisy(X_train_new,y_train_new,variational)
    return cost_function(X_train_new, y_train_new, variational)

### Definizione del logger per il tracciamento dell'ottimizzazione

Gestisce e salva i risultati intermedi dell’ottimizzazione durante l’addestramento del VQC. È progettata per essere usata come callback in `scipy.optimize.minimize`.

In [ ]:
class OptimizerLog:
    """Log per salvare i risultati intermedi dell'ottimizzazione."""
    def __init__(self):
        self.evaluations = 1
        self.parameters = []
        self.costs = []

    def callback(self, xk):
        """Funzione di callback compatibile con scipy.optimize.minimize."""
        cost = objective_function(xk)
        self.parameters.append(xk.copy())
        print("loss_function:", cost, "iterazione:", self.evaluations)
        self.costs.append(cost)
        self.evaluations += 1

### Avvio dell’ottimizzazione dei parametri del VQC

In [ ]:
logger = OptimizerLog()

initial_point = np.zeros((ansatz.num_parameters))
#initial_point = np.random.uniform(0, 2*np.pi, size=ansatz.num_parameters)

res = minimize(
    objective_function,
    initial_point,
    method="COBYLA", # ottimizzatore alternativo utilizzato COBYQA
    options={"maxiter": 5000},
    callback=logger.callback
)

In [ ]:
opt_var = res.x
opt_value = res.fun

fig = plt.figure()
plt.plot(logger.costs)
plt.xlabel('Steps')
plt.ylabel('Cost')
plt.show()

print("migliori parametri",opt_var)
print("miglior funzione obiettivo",opt_value)

### Funzione generale di valutazione del classificatore quantistico

La funzione `test_classifier` confronta ogni predizione con l’etichetta vera e calcola il numero di classificazioni corrette. Restituisce come risultato finale l’accuratezzae  la lista delle etichette predette.

In [ ]:
def test_classifier(data, labels, variational):

    probability = classification_probability(data, variational)
    #probability = classification_probability_with_noisy(data, variational)

    predictions= [max(p, key=p.get) for p in probability]
    accuracy = 0
    for i, prediction in enumerate(predictions):
        if prediction == labels[i]:
            accuracy += 1
    accuracy /= len(labels)
    return accuracy , predictions

### Valutazione del classificatore sui dati di training e test

In [ ]:
_ , predictions_train = test_classifier(X_train_new, y_train_new, opt_var)
_ , predictions_test = test_classifier(X_test_new, y_test_new, opt_var)

### Calcolo delle metriche di valutazione sul test

In [ ]:
accuracy  = accuracy_score(y_test_new, predictions_test)
precision = precision_score(y_test_new, predictions_test)
recall    = recall_score(y_test_new, predictions_test)
f1        = f1_score(y_test_new, predictions_test)

print(f"Accuracy_test: {accuracy:.2f}")
print(f"Precision_test: {precision:.2f}")
print(f"Recall_test: {recall:.2f}")
print(f"F1-score_test: {f1:.2f}")

### Confusion Matrix su train e test

In [ ]:
confusion_matrix(y_train_new,predictions_train)
confusion_matrix(y_test_new,predictions_test)